In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [14]:
EDU0_Raw = "../wonder/data_edu/Multiple Cause of Death, 2018-2021, Single Race (EDU_0-.1).txt"
EDU1_Raw = "data_edu/Multiple Cause of Death, 2018-2021, Single Race (EDU_.1-.2).txt"
EDU2_Raw = "data_edu/Multiple Cause of Death, 2018-2021, Single Race (EDU_.2-.3).txt"
EDU3_Raw = "data_edu/Multiple Cause of Death, 2018-2021, Single Race (EDU_.3-.4).txt"
EDU4_Raw = "data_edu/Multiple Cause of Death, 2018-2021, Single Race (EDU_.4-.5).txt"
EDU5_Raw = "data_edu/Multiple Cause of Death, 2018-2021, Single Race (EDU_.5-.6).txt"
EDU6_Raw = "data_edu/Multiple Cause of Death, 2018-2021, Single Race (EDU_.6-.7).txt"
EDU7_Raw = "data_edu/Multiple Cause of Death, 2018-2021, Single Race (EDU_.7-.8).txt"

In [31]:
EDU0 = pd.read_csv(EDU0_Raw, sep='\t', skipfooter=64, engine='python')
EDU1 = pd.read_csv(EDU1_Raw, sep='\t', skipfooter=64, engine='python')
EDU2 = pd.read_csv(EDU2_Raw, sep='\t', skipfooter=138, engine='python')
EDU3 = pd.read_csv(EDU3_Raw, sep='\t', skipfooter=128, engine='python')
EDU4 = pd.read_csv(EDU4_Raw, sep='\t', skipfooter=105, engine='python')
EDU5 = pd.read_csv(EDU5_Raw, sep='\t', skipfooter=138, engine='python')
EDU6 = pd.read_csv(EDU6_Raw, sep='\t', skipfooter=128, engine='python')
EDU7 = pd.read_csv(EDU7_Raw, sep='\t', skipfooter=105, engine='python')

In [32]:
EDU1

,Notes,Census Region,Census Region Code,Year,Year Code,Gender,Gender Code,Single Race 6,Single Race 6 Code,Five-Year Age Groups,Five-Year Age Groups Code,Deaths,Population,Crude Rate
0,NaN,Census Region 1: Northeast,CENS-R1,2018.0,2018.0,Female,F,White,2106-3,20-24 years,20-24,14.0,57092.0,Unreliable
1,NaN,Census Region 1: Northeast,CENS-R1,2018.0,2018.0,Female,F,White,2106-3,25-29 years,25-29,11.0,58859.0,Unreliable
2,NaN,Census Region 1: Northeast,CENS-R1,2018.0,2018.0,Female,F,White,2106-3,30-34 years,30-34,20.0,56732.0,35.3
3,NaN,Census Region 1: Northeast,CENS-R1,2018.0,2018.0,Female,F,White,2106-3,35-39 years,35-39,15.0,59301.0,Unreliable
4,NaN,Census Region 1: Northeast,CENS-R1,2018.0,2018.0,Female,F,White,2106-3,40-44 years,40-44,13.0,57882.0,Unreliable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743,"County, VA (51185); Westmoreland County, VA (5...",None,None,NaN,NaN,None,None,None,None,None,None,NaN,NaN,None
744,"(51580); Danville city, VA (51590); Emporia ci...",None,None,NaN,NaN,None,None,None,None,None,None,NaN,NaN,None
745,"VA (51670); Norton city, VA (51720); Petersbur...",None,None,NaN,NaN,None,None,None,None,None,None,NaN,NaN,None
746,"County, WA (53019); Franklin County, WA (53021...",None,None,NaN,NaN,None,None,None,None,None,None,NaN,NaN,None
